In [ ]:
import pandas as pd
from tabulate import tabulate
import smtplib
from email.message import EmailMessage
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Load the clients file
clientes_df = pd.read_csv("../data/clientes.csv", sep=";")  # Adjust the path if the notebook is at the same level


In [ ]:
# 📌 Email configuration
SMTP_SERVER =  "..."
SMTP_PORT = ...
EMAIL_SENDER =  "..."
EMAIL_PASSWORD =  "..."  # ⚠️ Better to load this from an environment variable instead of hardcoding

In [ ]:
def resaltar(valor, campo=""):
    if pd.isna(valor):
        return "<strong>No definido</strong>"

    # If numeric
    try:
        num = float(valor)
        if num.is_integer():
            num = int(num)
        else:
            num = round(num, 2)

        # Format with dots and add unit depending on the field
        if "precio" in campo:
            return f"<strong>{num:,}".replace(",", ".") + " €</strong>"
        elif "m2" in campo:
            return f"<strong>{num:,}".replace(",", ".") + " m²</strong>"
        elif "planta" in campo:
            return f"<strong>{num}</strong>"
        else:
            return f"<strong>{num}</strong>"

    except ValueError:
        pass  # Not a number

    # If text
    texto = str(valor).replace(",", ", ").replace("_", " ")
    texto = ' '.join(word.capitalize() for word in texto.strip().split())

    return f"<strong>{texto}</strong>"

def generar_email_html_por_correo(email_objetivo):
    cliente = clientes_df[clientes_df['email'] == email_objetivo].iloc[0]
    nombre = cliente['nombre']
    membresia = cliente.get("Membresía", "No definida").strip()

    membresia_lower = membresia.lower()

    # Customize intro depending on membership type
    if membresia_lower == "prueba gratuita":
        intro = "¡Gracias por comenzar tu <strong>prueba gratuita</strong>! Durante los próximos <strong>7 días</strong> recibirás informes automáticos con pisos que se ajustan a tus criterios y que representen buenas oportunidades."
    elif "mensual" in membresia_lower:
        intro = f"Has activado tu suscripción al <strong>{membresia}</strong>. Durante los próximos <strong>30 días</strong> recibirás informes automáticos con pisos que encajen con tus criterios y detectemos como buenas oportunidades."
    elif "semestral" in membresia_lower:
        intro = f"Has activado tu suscripción al <strong>{membresia}</strong>. Durante los próximos <strong>6 meses</strong> recibirás informes automáticos con pisos que cumplan tus filtros y representen oportunidades destacadas."

    # Build list of client specifications
    especificaciones = f"""
<ul>
    <li>Precio mínimo: {resaltar(cliente.get("precio_min"), "precio_min")}</li>
    <li>Precio máximo: {resaltar(cliente.get("precio_max"), "precio_max")}</li>
    <li>Metros mínimos: {resaltar(cliente.get("m2_min"), "m2_min")}</li>
    <li>Metros máximos: {resaltar(cliente.get("m2_max"), "m2_max")}</li>
    <li>Planta mínima: {resaltar(cliente.get("planta_min"), "planta_min")}</li>
    <li>Planta máxima: {resaltar(cliente.get("planta_max"), "planta_max")}</li>
    <li>Habitaciones mínimas: {resaltar(cliente.get("habitaciones_min"), "habitaciones_min")}</li>
    <li>Baños mínimos: {resaltar(cliente.get("baños_min"), "baños_min")}</li>
    <li>Exterior: {"<strong>Sí</strong>" if cliente.get("exterior") == 1 else "<strong>No o no definido</strong>"}</li>
    <li>Extras: {resaltar(cliente.get("extras"), "extras")}</li>
    <li>Estado vivienda: {resaltar(cliente.get("estados_vivienda"), "estado")}</li>
    <li>Barrios de interés: {resaltar(cliente.get("barrios_interes"), "barrios")}</li>
</ul>
"""

    # Final HTML template
    mensaje = f"""
<html>
  <body style="font-family: Arial, sans-serif; color: #000000; font-size: 15px; line-height: 1.5;">
    <p style="font-size: 18px;"><strong>¡Bienvenido a tu {membresia}!</strong></p>
    <p>Hola {nombre},</p>
    <p>{intro}</p>
    <p>🔍 Estas son las especificaciones que has definido:</p>
    {especificaciones}
    <p>Si deseas ajustar cualquier preferencia, puedes escribirnos a <a href="mailto:{EMAIL_SENDER}">{EMAIL_SENDER}</a> o responder a este correo.</p>
    <p>Gracias por confiar en nosotros.<br>El equipo de inmobil-IA-ria.</p>
  </body>
</html>
"""
    return mensaje

def enviar_email_bienvenida_html(destinatario, mensaje_html):
    msg = MIMEMultipart("alternative")
    msg["Subject"] = "Bienvenido a Inmobil-IA-ria"
    msg["From"] = EMAIL_SENDER
    msg["To"] = destinatario

    # Plain text fallback
    texto_plano = "Gracias por unirte a nuestro servicio. Si no puedes ver este mensaje, activa la vista HTML en tu correo."

    # Attach both plain text and HTML
    msg.attach(MIMEText(texto_plano, "plain"))
    msg.attach(MIMEText(mensaje_html, "html"))

    try:
        with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT) as smtp:
            smtp.login(EMAIL_SENDER, EMAIL_PASSWORD)
            smtp.send_message(msg)
        print(f"✅ Email successfully sent to {destinatario}")
    except Exception as e:
        print(f"❌ Error sending to {destinatario}: {e}")


In [ ]:
# 👤 Client email who just joined
email_cliente =  "..."

# Generate email content
mensaje_html = generar_email_html_por_correo(email_cliente)

# Send email
enviar_email_bienvenida_html(email_cliente, mensaje_html)
